```{contents}
```
## Session-Based Memory

### What Is Session-Based Memory

**Session-based memory** stores and retrieves conversation context **per user session**.
Each session has its **own isolated memory**, ensuring that conversations do not leak across users or browser sessions.

In LLM systems, session-based memory is a **scoping mechanism**, not a memory algorithm itself.

```
Session ID → Memory Instance → Conversation Context
```

It is commonly implemented using:

* Conversation memory (buffer / summary / window)
* A session key (session_id, user_id, chat_id)
* External storage (Redis, DB, in-process dict)

---

### Why Session-Based Memory Is Required

Without session-based memory:

* All users share the same context
* Conversations overwrite each other
* Multi-user chatbots break

With session-based memory:

* Each user gets a private context
* Parallel chats are supported
* Stateful UX is possible

---

### How Session-Based Memory Works

1. Client sends a request with `session_id`
2. Backend maps `session_id → memory object`
3. LLM uses only that memory
4. Memory is updated and stored back

```
User A (session_1) → Memory A
User B (session_2) → Memory B
```

---

### Architecture View

![Image](https://www.cognee.ai/content/blog/posts/from-demo-to-production-1/atkinson.png?utm_source=chatgpt.com)

![Image](https://miro.medium.com/v2/resize%3Afit%3A1400/1%2A1EbvDX7oUEQuzKyQ0a2AiA.png?utm_source=chatgpt.com)

![Image](https://miro.medium.com/v2/resize%3Afit%3A1400/1%2AKvbtCxuppbszOwcTFlS_Rw.png?utm_source=chatgpt.com)

---

### Session-Based Memory Using LangChain (In-Memory)

Using LangChain.

#### Session → Memory Store



In [1]:
from langchain_classic.memory import ConversationBufferMemory

session_store = {}

def get_memory(session_id: str):
    if session_id not in session_store:
        session_store[session_id] = ConversationBufferMemory()
    return session_store[session_id]




Each session gets its **own ConversationBufferMemory**.

---

### Attach Session Memory to a Chain



In [2]:
from langchain_classic.chains import ConversationChain
from langchain_openai import OpenAI

llm = OpenAI()

def get_conversation(session_id: str):
    memory = get_memory(session_id)
    return ConversationChain(llm=llm, memory=memory)



---

### Demonstration (Two Sessions)


In [3]:
chat_a = get_conversation("session_A")
chat_b = get_conversation("session_B")

chat_a.predict(input="My VPN is not working")
chat_b.predict(input="I cannot access email")

chat_a.predict(input="What issue did I report?")
chat_b.predict(input="What issue did I report?")

C:\Users\sangouda\AppData\Local\Temp\ipykernel_32856\518817146.py:7: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  session_store[session_id] = ConversationBufferMemory()
C:\Users\sangouda\AppData\Local\Temp\ipykernel_32856\767081169.py:8: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use `langchain_core.runnables.history.RunnableWithMessageHistory` instead.
  return ConversationChain(llm=llm, memory=memory)


" You reported not being able to access email. Is this a recurring issue or is it the first time it's happened?"

**Outputs**

```
Session A → You reported a VPN issue.
Session B → You reported an email access issue.
```

This proves **memory isolation per session**.

---

### Session-Based Memory with FastAPI



```python
from fastapi import FastAPI
from pydantic import BaseModel

app = FastAPI()

class ChatRequest(BaseModel):
    session_id: str
    message: str

@app.post("/chat")
def chat(req: ChatRequest):
    conversation = get_conversation(req.session_id)
    response = conversation.predict(input=req.message)
    return {"response": response}
```

Here:

* Frontend sends `session_id`
* Backend restores correct memory
* Chat remains stateful

---

### Session-Based Memory with Redis (Production)

```python
from langchain.memory import ConversationBufferMemory
from langchain.memory.chat_message_histories import RedisChatMessageHistory

def get_memory(session_id):
    history = RedisChatMessageHistory(
        session_id=session_id,
        url="redis://localhost:6379"
    )
    return ConversationBufferMemory(chat_memory=history)
```

Benefits:

* Survives restarts
* Horizontally scalable
* Multi-instance safe

---

### What Session-Based Memory Is NOT

| Misconception    | Reality                      |
| ---------------- | ---------------------------- |
| A memory type    | It is a **scoping strategy** |
| Long-term memory | Session-scoped               |
| Semantic recall  | Depends on memory used       |
| Model feature    | Application-level            |

---

### Session-Based Memory vs Memory Types

| Aspect  | Session-Based | Memory Type      |
| ------- | ------------- | ---------------- |
| Purpose | Isolation     | Recall strategy  |
| Scope   | Per user/chat | Global           |
| Example | session_id    | Buffer / Summary |
| Storage | Dict / Redis  | Text / Vectors   |

They are **orthogonal concepts**.

---

### Real-World Use Case

**Production IT Support Chatbot**

* session_id per browser tab
* Window memory for recent steps
* Entity memory for user environment
* Vector memory for long-term recall

All scoped by **session**.

---

### Best Practices

* Generate session IDs client-side
* Expire idle sessions
* Use Redis or DB in production
* Never share memory across sessions
* Log session → tool usage

---

### Key Takeaways

* Session-based memory ensures **conversation isolation**
* It wraps any memory type
* Mandatory for multi-user systems
* Foundation for scalable chat applications